<a href="https://colab.research.google.com/github/obijywk/grilops/blob/master/examples/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# grilops tutorial

This notebook will step through how to solve some logic puzzles using grilops and z3.

## Setup

First, we'll need to make sure the `grilops` package is installed. This will also install the `z3-solver` package if needed (as it is a dependency of `grilops`).

In [0]:
import sys
!{sys.executable} -m pip install grilops

     |████████████████████████████████| 19.1MB 6.4MB/s 


Next, we'll import the `grilops` module, and everything from the `z3` module (some consider wildcard imports to be an [anti-pattern](https://docs.quantifiedcode.com/python-anti-patterns/maintainability/from_module_import_all_used.html) in Python, but doing this is convenient for the purposes of this tutorial).

In [0]:
import grilops
from grilopos.geometry import Point
from z3 import *

Now we can move on to solving some puzzles!

## Sudoku

[Sudoku](https://en.wikipedia.org/wiki/Sudoku) is a good puzzle to start with, because it's well-known, and is relatively simple to model.

We'll start by creating a list of lists containing the pre-filled numbers given in the puzzle. We'll use the givens from the example puzzle from Wikipedia. We'll use a 0 to represent a cell for which we don't have a given value.

In [0]:
  givens = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
  ]

Now let's create a grilops `SymbolSet` to model the marks that we can fill into the grid (in this case, the digits 1 through 9), and a 9x9 grilops `SymbolGrid` to model the grid itself. See the grilops [Symbols](https://obijywk.github.io/grilops/symbols/) and [Grids](https://obijywk.github.io/grilops/grids/) documentation to learn more about these objects.

In [0]:
sym = grilops.make_number_range_symbol_set(1, 9)
locations = grilops.geometry.get_square_locations(9)
sg = grilops.SymbolGrid(locations, sym)

Our next step will be to enter our given numbers into the grid. We'll do this by looping over all of the positions in the grid, and constraining the grid to contain the given number at that position whenever it is not 0.

In [0]:
for y in range(9):
  for x in range(9):
    given = givens[y][x]
    if given != 0:
      sg.solver.add(sg.cell_is(Point(y, x), given))

When the `SymbolGrid` was constructed, it created a z3 [`Solver`](https://z3prover.github.io/api/html/classz3py_1_1_solver.html) object, accessible via its `solver` attribute. We'll use this solver to add all of our puzzle-specific constraints, and ultimately to solve the puzzle.

The `SymbolGrid.cell_is` method returns a constraint requiring that a cell at a given position in the grid contains a given symbol. Notice that the y (vertical) coordinate comes before the x (horizontal) coordinate; this matches the order we used to define our grid of givens, and is a convention used throughout grilops.

Next, let's add the defining constraints of Sudoku: each row, column, and 3x3 subgrid may only contain each digit one time. We'll use the z3 `Distinct` operator to express this.

In [0]:
rows = sg.grid
for row in rows:
  sg.solver.add(Distinct(*row))

columns = [[sg.grid[Point(y, x)] for y in range(9)] for x in range(9)]
for column in columns:
  sg.solver.add(Distinct(*column))

for subgrid_index in range(9):
  top = (subgrid_index // 3) * 3
  left = (subgrid_index % 3) * 3
  cells = [sg.grid[Point(y, x)] for y in range(top, top + 3) for x in range(left, left + 3)]
  sg.solver.add(Distinct(*cells))

Okay, we've added all of the constraints needed to model a Sudoku puzzle. Now let's try to solve it!

In [0]:
sg.solve()

True

`True` means we found a solution! Let's see what it is.

In [0]:
sg.print()

534678912
672195348
198342567
859761423
426853791
713924856
961537284
287419635
345286179


Looks good!

Let's check to see if there are any other possible solutions to this puzzle.

In [0]:
sg.is_unique()

True

This solution is unique. If it had turned out not to be unique (if there were an alternate solution) we could now call `sg.print()` again to see the alternate solution.

## Fillomino

Let's try a [Fillomino](https://en.wikipedia.org/wiki/Fillomino) puzzle now. This example will demonstrate the use of the grilops [`RegionConstrainer`](https://obijywk.github.io/grilops/regions/) to divide the grid into orthogonally contiguous regions of cells (polyominoes).

We'll start by creating a list of lists of the given region sizes, using 0 to indicate a cell that does not contain a given value.

In [0]:
givens = [
  [0, 0, 0, 3, 0, 0, 0, 0, 5],
  [0, 0, 8, 3, 10, 0, 0, 5, 0],
  [0, 3, 0, 0, 0, 4, 4, 0, 0],
  [1, 3, 0, 3, 0, 0, 2, 0, 0],
  [0, 2, 0, 0, 3, 0, 0, 2, 0],
  [0, 0, 2, 0, 0, 3, 0, 1, 3],
  [0, 0, 4, 4, 0, 0, 0, 3, 0],
  [0, 4, 0, 0, 4, 3, 3, 0, 0],
  [6, 0, 0, 0, 0, 1, 0, 0, 0],
]

Now we'll create our SymbolSet and our SymbolGrid.

In [0]:
sym = grilops.make_number_range_symbol_set(1, 10)
locations = grilops.geometry.get_square_locations(9)
sg = grilops.SymbolGrid(locations, sym)

Note that we're assuming that there will not be any region larger than 10 cells (the upper bound of our number range symbol set). We could make this upper bound arbitrarily large, but doing so might increase the search space, causing the solver to take longer to run.

Now we'll introduce a `RegionConstrainer` set up to use the same solver as our `SymbolGrid`. We'll need to import the `grilops.regions` module to use this class.

In [0]:
import grilops.regions
rc = grilops.regions.RegionConstrainer(locations, solver=sg.solver)

Okay, now we can start adding the constraints that define the logic of the puzzle.

First, we'll associate each symbol in the grid with the concept that its value represents: the size of the region to which the cell belongs. The `RegionConstrainer` provides us with a `region_size_grid` where each cell contains the size of that cell's region.

In [0]:
for y in range(9):
  for x in range(9):
    sg.solver.add(sg.grid[Point(y, x)] == rc.region_size_grid[Point(y, x)])

Next, we'll add a constraint for each of our givens ensuring that the size of the region matches the given's value.

In [0]:
for y in range(9):
  for x in range(9):
    given = givens[y][x]
    if given != 0:
      sg.solver.add(rc.region_size_grid[Point(y, x)] == given)

Finally, Fillomino requires that "no two polyominoes of matching size (number of cells) are orthogonally adjacent (share a side)." To add this constraint, we'll consider the orthogonal neighbors of each cell (see the `edge_sharing_neighbors` function and `Neighbor` class from the grilops [`grids`](https://obijywk.github.io/grilops/grids/) module to learn more about how these are found). We'll ensure that if two orthogonally adjacent cells have the same region size, that they are also part of the same region. We'll implement the "part of the same region" constraint using the `region_id_grid` attribute of the `RegionConstrainer`; in this grid, each cell will contain a numeric identifier that is shared among all cells that are part of the same region.

In [0]:
for y in range(9):
  for x in range(9):
    p = Point(y, x)
    adjacent_sizes = grilops.edge_sharing_neighbors(rc.region_size_grid, p)
    adjacent_ids = grilops.edge_sharing_neighbors(rc.region_id_grid, p)
    for adjacent_size, adjacent_id in zip(adjacent_sizes, adjacent_ids):
      sg.solver.add(
          Implies(
              rc.region_size_grid[p] == adjacent_size.symbol,
              rc.region_id_grid[p] == adjacent_id.symbol
          )
      )

And that's it! Time to solve.

In [0]:
sg.solve()

True

In [0]:
sg.print()

8 8 3 3 101010105 
8 8 8 3 1010105 5 
3 3 8 10104 4 4 5 
1 3 8 3 102 2 4 5 
2 2 8 3 3 1 3 2 2 
6 6 2 2 1 3 3 1 3 
6 4 4 4 2 2 1 3 3 
6 4 2 2 4 3 3 4 4 
6 6 4 4 4 1 3 4 4 


In [0]:
sg.is_unique()

True

## Akari

Now let's try solving an [Akari](https://en.wikipedia.org/wiki/Light_Up_%28puzzle%29) puzzle (also known as Light Up). This example will demonstrate the use of the grilops [`sightlines`](https://obijywk.github.io/grilops/sightlines/) module to check conditions along straight lines through the grid.

First, we'll encode the givens (the positions, and sometimes also the adjacent light bulb counts, of the black grid cells), using a Python dict. We'll use a value of `None` to indicate that a cell does not have an adjacent light bulb count constraint.

In [0]:
givens = {
  (0, 0): None,
  (0, 3): None,
  (0, 9): None,
  (1, 7): None,
  (2, 1): 3,
  (2, 6): 0,
  (3, 2): 2,
  (3, 5): None,
  (3, 9): 1,
  (4, 3): 1,
  (4, 4): 0,
  (4, 5): None,
  (5, 4): 1,
  (5, 5): None,
  (5, 6): None,
  (6, 0): None,
  (6, 4): 2,
  (6, 7): 2,
  (7, 3): None,
  (7, 8): None,
  (8, 2): 1,
  (9, 0): 0,
  (9, 6): 1,
  (9, 9): 0,
}

Next, we'll create the symbol set and the grid. We'll use three possible symbols in this grid: one to indicate a black cell, one to indicate an empty cell, and one to indicate a cell containing a light bulb. The `SymbolSet` constructor accepts a list of tuples, where the first element of each tuple contains a Python-safe attribute name for that symbol, and the second element of the tuple contains printable text used to represent that symbol in a grid printout.

In [0]:
height, width = 10, 10
sym = grilops.SymbolSet([
  ("BLACK", "#"),
  ("EMPTY", " "),
  ("LIGHT", "*"),
])
locations = grilops.geometry.get_rectangle_locations(height, width)
sg = grilops.SymbolGrid(locations, sym)

Now we'll start adding some constraints. The first set of constraints we'll model will be for the givens: if a cell was given at all, we'll ensure it's black, and if the given has a numeric value, we'll ensure the cells adjacent to it contain exactly that many light bulbs. In addition, we know that all black cells were given, so if a cell was not given we must constrain it to not be black.

In [0]:
for y in range(height):
  for x in range(width):
    p = Point(y, x)
    if (y, x) in givens:
      sg.solver.add(sg.cell_is(p, sym.BLACK))
      light_bulb_count = givens[(y, x)]
      if light_bulb_count is not None:
        sg.solver.add(light_bulb_count == sum(
            If(n.symbol == sym.LIGHT, 1, 0) for n in sg.edge_sharing_neighbors(p)
        ))
    else:
      # All black cells are given; don't allow this cell to be black.
      sg.solver.add(Not(sg.cell_is(p, sym.BLACK)))

The next set of constraints we'll add will enforce the general rules of the puzzle: every white cell must be lit (by having at least one light bulb in its row or column that is not blocked by a black cell), and light bulbs may not be visible to each other (if they are in the same row or column, there must be a black cell between them).

We'll use the `count_cells` function of the grilops [`sightlines`](https://obijywk.github.io/grilops/sightlines/) module to create these constraints. This function accepts as arguments a starting position in the grid, a direction to travel, and counting and stopping conditions for its travel, and returns the number of cells counted along the way.

We'll configure the counting condition to count the number of light bulbs encountered, and the stopping condition to become true when we reach a black cell. This will allow us to count the number of light bulbs visible in any direction from each cell, which we'll then use to set up both of the puzzle rule constraints mentioned above.

In [0]:
import grilops.sightlines

def is_black(c):
  return c == sym.BLACK

def count_light(c):
  return If(c == sym.LIGHT, 1, 0)

for y in range(height):
  for x in range(width):
    # Only add these visible light constraints for non-black cells.
    if (y, x) in givens:
      continue
    
    # For each cell adjacent to this one, count the visible cells in that
    # direction that contain light bulbs, then sum up all of these counts to
    # get the total number of visible light bulbs.
    visible_light_bulb_count = sum(
        grilops.sightlines.count_cells(
            sg, n.location, n.direction, stop=is_black, count=count_light
        ) for n in sg.edge_sharing_neighbors(Point(y, x))
    )
    
    # If this cell contains a light bulb, then ensure that it cannot see any
    # other cells that contain light bulbs. If this cell does not contain a
    # light bulb, then ensure that it is lit by ensuring that it can see at
    # least one light bulb.
    sg.solver.add(
      If(
          sg.cell_is(Point(y, x), sym.LIGHT),
          visible_light_bulb_count == 0,
          visible_light_bulb_count > 0
      )
    )

Those are all the constraints we need. Time to solve the puzzle.

In [0]:
sg.solve()

True

In [0]:
sg.print()

#* #*    #
   *   #  
*#*   #  *
 *#  #   #
   ###*   
   *###*  
# * #* #* 
*  #*   #*
  #     * 
# *   #* #


In [0]:
sg.is_unique()

True

## More Examples

The examples in this notebook are just a starting point. See the grilops [examples directory](https://github.com/obijywk/grilops/tree/master/examples) for standalone versions of these examples, as well as programs to solve many more types of puzzles.